数据集

In [1]:
import pandas as pd
import numpy as np

# 设置随机种子，确保可复现性
np.random.seed(42)

# 模拟数据的长度
n_regions = 10  # 假设有10个地区
n_years = 20  # 假设数据从2000年到2019年（20年）

# 生成地区列表和年份列表
regions = [f"Region_{i}" for i in range(1, n_regions + 1)]
years = list(range(2000, 2020))

# 创建模拟数据
data = []

for region in regions:
    for year in years:
        # 随机生成一些变量数据
        L = np.random.uniform(0.3, 0.8)  # 城市化进程（0到1之间）
        C = np.random.uniform(1000, 10000)  # 农村消费水平
        U = np.random.uniform(0.02, 0.1)  # 失业率
        Y = 5000 + 300 * L + np.random.normal(0, 1000)  # 农村收入（基于城市化进程生成，并加上噪音）
        
        # 将数据添加到列表中
        data.append([region, year, L, C, U, Y])

# 创建DataFrame
df = pd.DataFrame(data, columns=['region', 'year', 'L', 'C', 'U', 'Y'])

# 将数据保存为CSV文件
df.to_csv('rural_urban_data.csv', index=False)

print("数据文件 'rural_urban_data.csv' 已生成。")


数据文件 'rural_urban_data.csv' 已生成。


In [ ]:
import pandas as pd
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import compare
import numpy as np
import statsmodels.api as sm

# 加载数据
data = pd.read_csv("rural_urban_data.csv")

# 数据预处理：设置地区和年份为索引
data = data.set_index(["region", "year"])

# 面板数据回归模型（固定效应）
panel_data = data.copy()
model = PanelOLS.from_formula("Y ~ L + C + U + EntityEffects", data=panel_data)
result = model.fit()
print("固定效应回归结果:")
print(result.summary())

# 面板数据回归模型（随机效应）
re_model = RandomEffects.from_formula("Y ~ L + C + U", data=panel_data).fit()

# Hausman检验：比较固定效应和随机效应
# Hausman检验需要手动计算
# 使用差异的协方差矩阵进行Hausman检验
fe_params = fe_model.params
fe_cov = fe_model.cov
re_params = re_model.params
re_cov = re_model.cov

# 计算Hausman检验统计量
diff_params = fe_params - re_params
diff_cov = fe_cov - re_cov
hausman_stat = np.dot(np.dot(diff_params.T, np.linalg.inv(diff_cov)), diff_params)
hausman_pvalue = 1 - sm.stats.chisqprob(hausman_stat, len(fe_params))

print(f"Hausman检验统计量: {hausman_stat}")
print(f"Hausman检验p值: {hausman_pvalue}")


固定效应回归结果:
                          PanelOLS Estimation Summary                           
Dep. Variable:                      Y   R-squared:                        0.0078
Estimator:                   PanelOLS   R-squared (Between):             -0.0402
No. Observations:                 200   R-squared (Within):               0.0078
Date:                Tue, Dec 10 2024   R-squared (Overall):             -0.0388
Time:                        22:14:44   Log-likelihood                   -1648.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.4875
Entities:                          10   P-value                           0.6914
Avg Obs:                       20.000   Distribution:                   F(3,187)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             0.4875
                  